In [1]:
# import modules
# pseduo code
import rasterio
import shap
from rasterio.mask import mask
from rasterio.windows import from_bounds
from rasterio.transform import from_bounds 
import numpy as np
import sys
import os
import pickle
import subprocess
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.utils import register_keras_serializable
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Input, Add
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time
import matplotlib.pyplot as plt
from rasterio.transform import from_bounds
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
print("modules imported")

# file paths
Domain = "Sierras"
timestamp = "20250508_132928"
WorkspaceBase = f"D:/ASOML/{Domain}/"
modelOuptuts = WorkspaceBase + "modelOutputs/fromAlpine/"
savedModel = modelOuptuts + f"{timestamp}/best_model_{timestamp}.keras"
shapSavedArrays = WorkspaceBase + "SHAP_arrays/"

modules imported


In [2]:
# load in custom loss functions
@register_keras_serializable()
def masked_loss_fn(y_true, y_pred, loss_fn, mask_value=-1):
    mask = tf.not_equal(y_true, mask_value)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return loss_fn(y_true_masked, y_pred_masked)

@register_keras_serializable(name="masked_mse")
def masked_mse(y_true, y_pred):
    return masked_loss_fn(y_true, y_pred, MeanSquaredError())

@register_keras_serializable(name="masked_mae")
def masked_mae(y_true, y_pred):
    return masked_loss_fn(y_true, y_pred, MeanAbsoluteError())

@register_keras_serializable(name="masked_rmse")
def masked_rmse(y_true, y_pred):
    mse = masked_loss_fn(y_true, y_pred, MeanSquaredError())
    return tf.sqrt(mse)

def constrained_mse(fsca_channel_index):
    def loss(y_true, y_pred):
        mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)

        # Reshape predictions back to (batch_size, 256, 256)
        y_pred_reshaped = tf.reshape(y_pred, (-1, 256, 256))

        # Grab fSCA input from the input tensor (assumes you feed it into model.fit)
        fsca_input = tf.keras.backend.placeholder(shape=(None, 256, 256, 18))  # you’ll override this in fit

        fsca = fsca_input[..., fsca_channel_index]

        # Create a mask for where fSCA > 0 but y_pred <= 0
        violation_mask = tf.logical_and(fsca > 0, y_pred_reshaped <= 0)
        violation_penalty = tf.cast(violation_mask, tf.float32) * 100.0  # 100 is an arbitrary penalty factor

        penalty = tf.reduce_mean(violation_penalty, axis=[1, 2])  # penalize across spatial dimensions
        return mse + penalty
    return loss

In [3]:
# load in model, arrays, and list
model = tf.keras.models.load_model(savedModel)

# Load arrays
X_train = np.load(f'{shapSavedArrays}/X_train.npy')
X_valid = np.load(f'{shapSavedArrays}/X_valid.npy')
# Load other arrays if needed
y_train = np.load(f'{shapSavedArrays}/y_train.npy')
y_valid = np.load(f'{shapSavedArrays}/y_valid.npy')

# Load featureName list
with open(f'{shapSavedArrays}/featureName.pkl', 'rb') as f:
    featureName = pickle.load(f)

In [4]:
print("SHAP: running SHAP")
        
# Steps 1-4 as in the previous code
background_data = X_train
print("####background data established")
explainer = shap.GradientExplainer(model, background_data)
print("####explainer established")
examples_to_explain = X_valid
shap_values = explainer.shap_values(examples_to_explain)
print("####shap values determined")

# Calculate mean absolute SHAP value for each channel
channel_importance = np.abs(shap_values[0]).mean(axis=(0, 1, 2))
print("####calculating channel importances")

# Create channel importance plot with feature names
plt.figure(figsize=(14, 7))
bars = plt.bar(range(18), channel_importance)
plt.xlabel('Channels', fontsize=12)
plt.ylabel('Mean |SHAP value|', fontsize=12)
plt.title('Channel Importance by Feature', fontsize=14)
print("####channel importance created")

# Use the feature names as x-tick labels
plt.xticks(range(18), featureName, rotation=45, ha='right', fontsize=10)

# Add values on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.002,
             f'{height:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(shapSavedArrays + 'channel_importance_with_features.png', dpi=300)
plt.show()

# You can also create a sorted version to see the most important features first
sorted_idx = np.argsort(channel_importance)[::-1]  
sorted_importance = channel_importance[sorted_idx]
sorted_features = [featureName[i] for i in sorted_idx]

plt.figure(figsize=(14, 7))
bars = plt.bar(range(18), sorted_importance)
plt.xlabel('Channels', fontsize=12)
plt.ylabel('Mean |SHAP value|', fontsize=12)
plt.title('Channel Importance by Feature (Sorted)', fontsize=14)
plt.xticks(range(18), sorted_features, rotation=45, ha='right', fontsize=10)

# Add values on top of each bar
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.002,
             f'{height:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(shapSavedArrays + 'sorted_channel_importance.png', dpi=300)
plt.show()

SHAP: running SHAP
####background data established
####explainer established


C:\Users\etyrr\anaconda3\envs\aso-dl\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


MemoryError: Unable to allocate 900. MiB for an array with shape (200, 256, 256, 18) and data type float32